In [1]:
import os
from pathlib import Path
from monai.apps.auto3dseg import AutoRunner
from monai.config import print_config
import shutil

print_config()

MONAI version: 1.4.0
Numpy version: 1.26.4
Pytorch version: 2.5.1+cu124
MONAI flags: HAS_EXT = False, USE_COMPILED = False, USE_META_DICT = False
MONAI rev id: 46a5272196a6c2590ca2589029eed8e4d56ff008
MONAI __file__: /home/<username>/.virtualenvs/monai/lib/python3.12/site-packages/monai/__init__.py

Optional dependencies:
Pytorch Ignite version: NOT INSTALLED or UNKNOWN VERSION.
ITK version: 5.4.0
Nibabel version: 5.3.2
scikit-image version: 0.24.0
scipy version: 1.14.1
Pillow version: 11.0.0
Tensorboard version: 2.18.0
gdown version: 5.2.0
TorchVision version: 0.20.1+cu124
tqdm version: 4.66.6
lmdb version: 1.5.1
psutil version: 6.1.0
pandas version: 2.2.3
einops version: 0.8.0
transformers version: 4.46.2
mlflow version: 2.17.2
pynrrd version: 1.1.1
clearml version: 1.16.5

For details about installing the optional dependencies, please visit:
    https://docs.monai.io/en/latest/installation.html#installing-the-recommended-dependencies



In [ ]:
dataroot = Path("/media/smbshare/srs-9/prl_project/training/test_train2")
training_work_home = Path("/home/srs-9/Projects/prl_project/training_work_dirs")
work_dir = training_work_home / "test_train2_segresnet" #! edit this
datalist_file_src = os.path.join(os.getcwd(), "datalist.json")
datalist_file = work_dir / "datalist.json"
algos = ["segresnet"]

description = """Model: SegResNet
Trained using flair.phase.t1 images. 
Labels by Lia, edited to make 5->1, 4->1 (extra T2L and iron content as red)"""

Check that every file in the datalist exists

In [6]:
import json

with open(datalist_file_src, 'r') as f:
    datalist = json.load(f)

for item in datalist['training']:
    assert os.path.exists(item['image'])
    assert os.path.exists(item['label'])

for item in datalist['testing']:
    assert os.path.exists(item['image'])

Create the work_dir, copy over the datalist, and save an info file

In [7]:
if not work_dir.exists():
    print(f"Creating {str(work_dir)}")
    os.makedirs(work_dir)

shutil.copyfile(datalist_file_src, datalist_file)

with open(work_dir / "info.txt", 'w') as f:
    f.write(description)

print("work_dir is: ", work_dir)

Creating /home/srs-9/Projects/prl_project/training_work_dirs/test_train2_segresnet
work_dir is:  /home/srs-9/Projects/prl_project/training_work_dirs/test_train2_segresnet


In [9]:
runner = AutoRunner(
    work_dir=work_dir,
    algos=algos,
    input={
        "modality": "MRI",
        "datalist": str(datalist_file),
        "dataroot": str(dataroot),
    },
)

2025-08-20 16:15:23,917 - INFO - AutoRunner using work directory /home/srs-9/Projects/prl_project/training_work_dirs/test_train2_segresnet
2025-08-20 16:15:23,918 - INFO - Found num_fold 2 based on the input datalist /home/srs-9/Projects/prl_project/training_work_dirs/test_train2_segresnet/datalist.json.
2025-08-20 16:15:23,919 - INFO - Setting num_fold 2 based on the input datalist /home/srs-9/Projects/prl_project/training_work_dirs/test_train2_segresnet/datalist.json.
2025-08-20 16:15:23,920 - INFO - Using user defined command running prefix , will override other settings


In [10]:
max_epochs = 250

train_param = {
    "num_epochs_per_validation": 1,
    #"num_images_per_batch": 2,
    "num_epochs": max_epochs,
    "num_warmup_epochs": 1,
}
runner.set_training_params(train_param)

In [11]:
runner.run()

2025-08-20 16:15:32,389 - INFO - Running data analysis...
2025-08-20 16:15:32,390 - INFO - Found 1 GPUs for data analyzing!


100%|██████████| 10/10 [00:17<00:00,  1.70s/it]

2025-08-20 16:15:49,503 - INFO - Data spacing is not completely uniform. MONAI transforms may provide unexpected result
2025-08-20 16:15:49,503 - INFO - Writing data stats to /home/srs-9/Projects/prl_project/training_work_dirs/test_train2_segresnet/datastats.yaml.
2025-08-20 16:15:49,509 - INFO - Writing by-case data stats to /home/srs-9/Projects/prl_project/training_work_dirs/test_train2_segresnet/datastats_by_case.yaml, this may take a while.
2025-08-20 16:15:49,583 - INFO - BundleGen from https://github.com/Project-MONAI/research-contributions/releases/download/algo_templates/e4cf5a1.tar.gz



algo_templates.tar.gz: 104kB [00:00, 461kB/s]                              

2025-08-20 16:15:49,816 - INFO - Downloaded: /tmp/tmpt3gkfbtg/algo_templates.tar.gz
2025-08-20 16:15:49,816 - INFO - Expected md5 is None, skip md5 check for file /tmp/tmpt3gkfbtg/algo_templates.tar.gz.
2025-08-20 16:15:49,816 - INFO - Writing into directory: /home/srs-9/Projects/prl_project/training_work_dirs/test_train2_segresnet.


2025-08-20 16:15:49,855 - INFO - Generated:/home/srs-9/Projects/prl_project/training_work_dirs/test_train2_segresnet/segresnet_0
2025-08-20 16:15:49,875 - INFO - Generated:/home/srs-9/Projects/prl_project/training_work_dirs/test_train2_segresnet/segresnet_1
2025-08-20 16:15:49,888 - INFO - ['python', '/home/srs-9/Projects/prl_project/training_work_dirs/test_train2_segresnet/segresnet_0/scripts/train.py', 'run', "--config_file='/home/srs-9/Projects/prl_project/training_work_dirs/test_train2_segresnet/segresnet_0/configs/hyper_parameters.yaml'", '--num_epochs_per_validation=1', '--num_epochs=250', '--num_warmup_epochs=1']


`torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
2025/08/20 16:15:53 INFO mlflow.tracking.fluent: Experiment with name 'Auto3DSeg' does not exist. Creating a new experiment.
segresnet_0 - training:   0%|          | 0/250 [00:00<?, ?epoch/s]`torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
`torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
segresnet_0 - training:  25%|██▌       | 63/250 [12:22<36:43, 11.78s/epoch] 
Traceback (most recent call last):
  File "/home/srs-9/Projects/prl_project/training_work_dirs/test_train2_segresnet/segresnet_0/scripts/train.py", line 27, in <module>
    fire.Fire()
  File "/home/srs-9/.virtualenvs/monai/lib/python3.12/site-packages/fire/core.py", line 135, in Fire
    component_trace = _Fire(component, args, parsed_flag_args, context, name)
                      ^^^^^^^^^^^^^^^^^^

CalledProcessError: Command '['python', '/home/srs-9/Projects/prl_project/training_work_dirs/test_train2_segresnet/segresnet_0/scripts/train.py', 'run', "--config_file='/home/srs-9/Projects/prl_project/training_work_dirs/test_train2_segresnet/segresnet_0/configs/hyper_parameters.yaml'", '--num_epochs_per_validation=1', '--num_epochs=250', '--num_warmup_epochs=1']' returned non-zero exit status 1.